This model is the classifier for the original datasets. Remember to check that you have filtering setting the way you want it :)

In [1]:
from scipy.io import loadmat
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymatreader import read_mat
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import eeg_funcs

In [ ]:
# Preprocess and cut the BCI III data in to samples and form that in to a numpy array
# labels in a seperate array

# check the filepath!
# remember to change the filtering to True/False
filepath_3 = glob('GANs/data/Three/Train/*.mat')
d_array, d_labels = eeg_funcs.get_data_for_classifier_comp3(filepath_3, False, True, True)
print(d_array.shape)
print(d_labels.shape)

Classifier Modeling

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

In [4]:
# train and validation split for the data

X_train, X_val, y_train, y_val = train_test_split(d_array, d_labels, train_size=0.8)
print(X_train.shape)

(40571, 160, 8)


In [ ]:
# classifier model 

model = keras.Sequential(
    [
        keras.Input(shape=X_train.shape[1:]),
        layers.BatchNormalization(),
        layers.Conv1D(16, kernel_size=1, strides=1, padding="same"),
        layers.Conv1D(16, kernel_size=20, strides=20, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation="tanh"),
        layers.Dropout(0.2),
        layers.Dense(128, activation="tanh"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

In [6]:
#compile the model
# playing around with the optimizer parameters possible

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics="accuracy",
    )

In [ ]:
#train the model
# playing around with the batch size and epochs possible

model.fit(
    x=X_train,
    y=y_train,
    batch_size=64,
    epochs=65,
    validation_data=(X_val,y_val),
    shuffle=True
)

In [ ]:
#putting the results of the latest training cycle in to a confusion matrix

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_prediction = model.predict(X_val)
y_prediction = y_prediction.flatten()
y_prediction = np.where(y_prediction > 0.5, 1, 0)

result = confusion_matrix(y_val, y_prediction, normalize='all')
print(result)
ConfusionMatrixDisplay(result).plot()

#change a correct file location!
plt.savefig("C:/Users/telab/Downloads/datafiles/pictures/CMatrix_Fil_Copies.png")

Evaluation of model on BCI 2 data

In [ ]:
# Getting the BCI II competition data for final classification test
# remember to check filepath!
# remember to change the filtering to True/False
filepath_2 = glob('GANs/data/train/*.mat')
X_test, y_test = eeg_funcs.get_data_for_classifier_comp2(filepath_2, False, True, False)

In [ ]:
#evaluation on the BCI II data

model.evaluate(
    x = X_test,
    y = y_test,
    batch_size = 64
)

In [ ]:
#Drawing confusion matrix from the BCI II test

t_prediction = model.predict(X_test)
t_prediction = t_prediction.flatten()
test_t = t_prediction.round(2)
t_prediction = np.where(t_prediction > 0.5, 1, 0)

result = confusion_matrix(y_test, t_prediction, normalize='all')
print(result)
ConfusionMatrixDisplay(result).plot()
plt.savefig("tempC.png")